In [1]:
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.time import BaseScheduler

from mesa.space import MultiGrid
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule, TextElement
from mesa.datacollection import DataCollector

In [2]:
# To calculate number of walking customers vs driving customers

def walking_customers(population, walk_ratio):
    walking_customers = int(population*walk_ratio)
    return walking_customers

### Model

In [11]:
class OutdoorDining(Model):
    
    """A model about outdoor dining spaces vs parking with regards to restaurants' revenues"""
    
    def __init__(self, width=20, height=10, population=100, 
                 eatout_prob=0.5, walk_ratio=0.4, RestA=False, RestB=False, taken_parking_prob = 0.3):
        super().__init__()
        self.running = True
        self.width = width
        self.height = height
        self.population = population
        
        self.restaurants_list = []
        self.parking_list = []
        self.RestA_outdoor = RestA
        self.RestB_outdoor = RestB
        self.eatout_prob = eatout_prob
        self.walk_ratio = walk_ratio
        
        self.total_walkers = walking_customers(self.population, self.walk_ratio)
        self.total_drivers = self.population - self.total_walkers
        
        self.grid = MultiGrid(width, height, False)
        self.schedule = RandomActivation(self)
        grid_map = np.genfromtxt("city.txt")
        
            
        #other buildings/homes (#0 for drivers or #7 for walkers)

        # creating restaurant A (#1)
        for _,x, y in self.grid.coord_iter():
            if grid_map[x,y] == 1: 
                block = Restaurant((x,y), self, name = 'Restaurant_A', outdoor=self.RestA_outdoor, tables=7)
                self.grid.place_agent(block,(x, y))
                self.schedule.add(block)
                self.restaurants_list.append(block)
                
        # creating parking spots if outdoor eating space for Restaurant A is False (#2)
        for _,x, y in self.grid.coord_iter():
            if (grid_map[x,y] == 2) and (self.RestA_outdoor == False): 
                block = Parking((x,y), self, taken_parking_prob)
                self.grid.place_agent(block,(x, y))
                self.schedule.add(block)
                self.parking_list.append(block)
        
        # creating restaurant B (#4)
        for _,x, y in self.grid.coord_iter():
            if grid_map[x,y] == 4: 
                block = Restaurant((x,y), self, name = 'Restaurant_B', outdoor=self.RestB_outdoor, tables=7)
                self.grid.place_agent(block,(x, y))
                self.schedule.add(block)
                self.restaurants_list.append(block)
                
        # creating parking spots if outdoor eating space for Restaurant B is False (#5)
        for _,x, y in self.grid.coord_iter():
            if (grid_map[x,y] == 5) and (self.RestB_outdoor == False): 
                block = Parking((x,y), self, taken_parking_prob)
                self.grid.place_agent(block,(x, y))
                self.schedule.add(block)  
                self.parking_list.append(block)
                
        # creating parking spots (#3)
        for _,x, y in self.grid.coord_iter():
            if grid_map[x,y] == 3: 
                block = Parking((x,y), self, taken_parking_prob)
                self.grid.place_agent(block,(x, y))
                self.schedule.add(block)
                self.parking_list.append(block)
            
    
        driver_coords = [(x,y) for _, x, y in self.grid.coord_iter() if grid_map[x,y] == 0]
        walker_coords = [(x,y) for _, x, y in self.grid.coord_iter() if grid_map[x,y] == 7]
        
        #create walking customers
        for i in range(self.total_walkers):            
            home = random.choice(walker_coords)
            a = Walking_Customer(i, self, home)
            self.schedule.add(a)
            self.grid.place_agent(a, home)
           
        #create driving customers
        for i in range(200,200+self.total_drivers):            
            home = random.choice(driver_coords)
            a = Driving_Customer(i, self, home)
            self.schedule.add(a) 
            self.grid.place_agent(a, home)            
            
        
        #data
        self.datacollector = DataCollector(
            model_reporters={"Restaurant A Revenue": get_RevenueA,
                             "Restaurant B Revenue": get_RevenueB}
            )       
             
    
    def step(self):
        self.schedule.step()   
        self.datacollector.collect(self)

### Customers

In [4]:
class Customer(Agent):

    def __init__(self, unique_id, model, home):
        super().__init__(unique_id, model)
        self.home = home
        self.pos = home # Customer is initialised at home!
        self.prob_eat = self.model.eatout_prob
        self.party = random.choice(range(1,9))
        self.time_till_eat = 0  
        self.poss_times_eating = [x / 2 for x in range(1, 4)]
        self.eating_duration = 0
        self.rest_pick = None
        self.outdoor_preference = random.choice([x / 10 for x in range(0, 10)]) #customer's OD preference
    
    def pick_restaurant(self):
        restaurant_weights = []
        for r in self.model.restaurants_list:
            w = r.outdoor
            if w*1 == 1 : 
                restaurant_weights.append(round(1 + self.outdoor_preference, 1))
            else: restaurant_weights.append(round(1 - self.outdoor_preference, 1))

        t = [i / sum(restaurant_weights) for i in restaurant_weights]
        pick = random.choices(['Restaurant_A', 'Restaurant_B'], weights=(t), k=1)[0]
        rest_pick = [r for r in self.model.restaurants_list if r.name == pick][0]
        self.rest_pick = rest_pick
    
    #function for leaving restaurant after eating there
    def leave(self):
        tables_needed = math.ceil(self.party/4)
        rest = [r for r in self.model.restaurants_list if r.name == self.rest_pick.name]
        for r in rest:
            r.tables_open = r.tables_open + tables_needed # increase restaurant capacity
        self.model.grid.move_agent(self, self.home) # move the agent back to home position
        self.rest_pick = None

### Walking Customers

In [5]:
class Walking_Customer(Customer):

    def __init__(self, unique_id, model, home):
        super().__init__(unique_id, model, home)        
         
    def walk_eat(self):
        tables_needed = math.ceil(self.party/4)
        
        if self.rest_pick.tables_open >= tables_needed: #customer can only go to restaurant if a table is open
                
            self.model.grid.move_agent(self, self.rest_pick.pos) # move customer to restaurant location 
            ''''''
            rest = [r for r in self.model.restaurants_list if r.name == self.rest_pick.name]
            for r in rest:
                r.tables_open = r.tables_open - tables_needed # reduce restaurant's number of open tables
            self.eating_duration = random.choice(self.poss_times_eating)  #length of eating time at restaurant   
            self.time_till_eat = 4.0 # it will be another 4 hrs before they consider eating again

        
    def step(self):
        if (self.time_till_eat <= 0):
            if (random.uniform(0, 1) < self.prob_eat): # decision to eat out or not
                self.pick_restaurant()
                self.walk_eat()
        else:
            # at every time step reduce amount of time left in the restaurant
            if self.eating_duration > 0: 
                self.eating_duration -= .5
                
            # at every step reduce the amount of time left before eating again
            if self.time_till_eat > 0: 
                self.time_till_eat -= .5  
                
            # if they have finished eating they leave restaurant
            if (self.rest_pick is not None) and (self.eating_duration <= 0): 
                self.leave()                         

### Driving Customers

In [6]:
class Driving_Customer(Customer):

    def __init__(self, unique_id, model, home):
        super().__init__(unique_id, model, home)
        self.parking_spot = None
         
    def drive_park_eat(self):
        tables_needed = math.ceil(self.party/4)
        
        if self.rest_pick.tables_open >= tables_needed: #customer can only go to restaurant if a table is open
            open_parking = [p for p in self.model.parking_list if p.available == True] 
            if len(open_parking) > 0:
                self.parking_spot = open_parking[0]
                
                (open_parking[0]).available = False
                self.eating_duration = random.choice(self.poss_times_eating) #length of time eating at restaurant
                (open_parking[0]).parking_duration = self.eating_duration #set the parking time = eating time
                
                self.model.grid.move_agent(self, self.rest_pick.pos) # move customer to restaurant
                
                rest = [r for r in self.model.restaurants_list if r.name == self.rest_pick.name]
                for r in rest:
                    r.tables_open = r.tables_open - tables_needed # reduce restaurant's number of open tables
             
                self.time_till_eat = 4.0 # it will be another 4 hrs before they consider eating again
    
    def step(self):
        if (self.time_till_eat <= 0):
            if (random.uniform(0, 1) < self.prob_eat): # decision to eat out or not
                self.pick_restaurant()
                self.drive_park_eat()
        else:
            # at every time step reduce amount of time left in the restaurant
            if self.eating_duration > 0: 
                self.eating_duration -= .5
                
            # at every step reduce the amount of time left before eating again
            if self.time_till_eat > 0: 
                self.time_till_eat -= .5  
                
            # if they have finished eating they leave restaurant
            if (self.rest_pick is not None) and (self.eating_duration <= 0): 
                self.leave()  

### Restaurants

In [7]:
class Restaurant(Agent):

    def __init__(self, pos, model, name, outdoor, tables):
        super().__init__(pos, model)
        self.pos = pos
        x,y = self.pos
        self.name = name
        self.outdoor = outdoor
        self.indoor_tables = tables
        self.tables_open = self.indoor_tables + (3*self.outdoor)
        self.revenue = 0
        self.price = 10
    
    def serve(self):
        customers = self.model.grid.get_cell_list_contents([self.pos])  # list of all agents in the cell
        if len(customers) > 1:
            for customer in customers:
                if isinstance(customer, Customer):  # if the agent in the cell is a Customer object...
                    self.revenue = self.revenue + (customer.party * self.price)    
    
    def step(self):
        self.serve()

### Parking Spots

In [8]:
class Parking(Agent):

    def __init__(self, pos, model, taken_parking_prob):
        super().__init__(pos, model)
        self.taken_parking_prob = taken_parking_prob
        self.available = True
        self.poss_times_parking = [x / 2 for x in range(1, 5)]
        self.parking_duration = 0
     
    def check_availability(self):
        self.parking_duration = random.choice(self.poss_times_parking)
        self.available = False

     
    def step(self):
        if (self.available == True):
            if (random.uniform(0,1) < self.taken_parking_prob):
                self.check_availability()
        else:
            if (self.available == False) and (self.parking_duration > 0):
                self.parking_duration -= .5
                
            if self.parking_duration <= 0:
                self.available = True

### Data Collection

In [9]:
def get_RevenueA(model):
    resta = [agent for agent in model.schedule.agents if (type(agent) is Restaurant) and (agent.name == 'Restaurant_A')]
    return resta[0].revenue

def get_RevenueB(model):
    restb = [agent for agent in model.schedule.agents if (type(agent) is Restaurant) and (agent.name == 'Restaurant_B')]
    return restb[0].revenue

### Agent Protrayal

In [10]:
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {}
    
    if type(agent) is Driving_Customer :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5, "Color" : "#f5b342", "Layer":1,}
    
    if type(agent) is Walking_Customer :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.35, "Color" : "#4cc279", "Layer":1,}
      
    
    if type(agent) is Restaurant :
        portrayal = {"Shape": "rect",
                 "Filled": "true",
                "Layer": 0 , "w":1 , "h":1, "Color": 'purple'}
    elif type(agent) is Parking:
        
        portrayal = {"Shape": "rect",
                 "Filled": "true",
                "Layer": 0 , "w":1 , "h":1,}
        
        if agent.available== True:
            portrayal["Color"] = "#d1d1d1"
        else:
            portrayal["Color"] = "#b3ae8d" 
               

    return portrayal



grid = CanvasGrid(agent_portrayal, 20, 10, 690, 345)
model_params = {
    "height": 10,
    "width": 20,
    "RestA": UserSettableParameter("checkbox", "Restaurant A Outdoor Dining", value=False),
    "RestB": UserSettableParameter("checkbox", "Restaurant B Outdoor Dining", value=False),
    "population": UserSettableParameter("slider", "Population", 100, 50, 250, 10),
    "eatout_prob": UserSettableParameter("slider", "Probability of Eating Out", 0.4, 0.1, 0.9, 0.1),
    "walk_ratio": UserSettableParameter("slider", "Ratio of Walkers to Drivers", 0.4, 0.2, 0.8, 0.1),
    "taken_parking_prob": UserSettableParameter("slider", "Probability Parking is Taken", 0.5, 0.2, 0.9, 0.1)
    
}

/Users/ayinaanyachebelu/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/mesa/visualization/UserParam.py:71: UserWarning: UserSettableParameter is deprecated in favor of UserParam objects such as Slider, Checkbox, Choice, StaticText, NumberInput. See the examples folder for how to use them. UserSettableParameter will be removed in the next major release.
  warn(


### Server

In [12]:
port_num=8521

In [13]:
#port_num+=1

Revenue = ChartModule([{"Label": "Restaurant A Revenue",
                      "Color": "red"},{"Label": "Restaurant B Revenue",
                      "Color": "blue"}],
                    data_collector_name='datacollector')


server = ModularServer(OutdoorDining,
                       [grid, Revenue], 
                       "Outdoor Dining Model",
                       model_params)

server.port = port_num
server.launch()

Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"submit_params","param":"RestA","value":true}
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type